<a href="https://colab.research.google.com/github/chongzicbo/nlp-ml-dl-notes/blob/master/%E7%9F%A5%E4%B9%8E%E7%AC%94%E8%AE%B0%EF%BC%9Abert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



##1.   BERT(Bidirectional Encoder Representations from Transformers)
BERT本质是通过海量的语料运行自监督学习方法为单词学习一个好的特征表示。在以后特定的NLP任务中可以直接使用BERT的特征表示作为该任务的词嵌入特征。所以BERT提供的是一个供其他任务迁移学习的模型，该模型可以根据任务微调或者固定之后作为特征提取器。
##2.   主要特点


*   使用Transformer作为算法主要框架，能够更加彻底地捕捉句中的双向关系；
*   使用了Mask Language Model（MLM）和Next Setence Prediction(NSP)的多任务训练目标；
*   使用更强大的机器训练更大规模的数据，使BERT的结果达到全新的高度。用户可以直接使用BERT作为Word2Vec的转换矩阵并高效的将其应用到自己的任务中。

##3. 网络架构

<img src="https://pic1.zhimg.com/80/v2-583765858249a8ebf5c0ececea2cd63c_720w.jpg" width="300" />

BERT的网络架构使用的是多层Transformer结构，上图中的左侧部分是一个Transformer Block,对应下图中的一个Trm

<img src="https://pic1.zhimg.com/80/v2-9979c95d66a71a720207a48311702430_720w.jpg" width="300" />

BERT提供了简单和复杂模型，对应的超参数如下：
* $BERT _{BASE}$:L=12,H=768,A=12,参数总量110M
* $BERT _{LARGE}$:L=24,H=1024,A=16,参数总量340M
其中L表示网络的 层数（即Transformer blocks的数量）,A表示Multi-Head Attention中self-attention的数量,filter的大小为4H。

* BERT与GPT和ELMO

<img src="https://pic1.zhimg.com/80/v2-d36db30f3c68a097f32ed0b49f9845cc_720w.jpg" width="500" />

与GPT和ELMO算法相比，BERT的表征会基于所有层中的左右两侧语境，BERT能够做到这一点得益于Transformer中的Attention机制，其会将任意位置的两个单词距离转换成1.

### 4. BERT的输入表示
BERT的输入编码向量（512维）是3个嵌入特征的单位和：
* WordPiece嵌入：WordPiece是指将单词划分成一组有限的公共子词单元，能在单词的有效性和字符的灵活性之间取得一个折中的平衡。playing ->play和ing
* Position Embedding：位置嵌入是指将单词的位置信息编码成特征向量，位置嵌入是向模型引入单词位置关系的至关重要的一环。
* Segment Embedding:用于区分两个句子，例如B是否是A的下文（对话场景，问答场景等）。对于句子对，第一个句子的特征值是0，第二个句子的特征值是1。

<img src="https://pic4.zhimg.com/80/v2-4f9f62a7776afcdd1e1c99dfa57b965f_720w.jpg" width="500" />

[CLS]和[SEP]，其中[CLS]表示该特征用于分类模型，对非分类模型，该符合可以省去。[SEP]表示分句符号，用于断开输入语料中的两个句子。

## 5. 预训练任务

### 5.1 Mask Language Model(MLM)
所谓MLM是指在训练的时候随机从输入语料中mask掉一些单词，然后通过上下文预测该单词。\
在BERT中，15%的Word Piece Token会被随机Mask掉，在训练模型时，一个句子会被多次输入模型中用于参数学习，但是不是每次都mask掉这些单词，而是在确定要Mask掉的单词之后，80%的概率该单词会替换为[Mask]，10%的概率会替换为其他任意单词，10%的概率会保留原始Token
* 80%：my dog is hairy -> my dog is [mask]
* 10%：my dog is hairy -> my dog is apple
* 10%：my dog is hairy -> my dog is hairy

这么做的原因是如果句子中的某个Token100%都会被mask掉，那么在fine-tuning的时候模型就会有一些没有见过的单词。加入随机Token的原因是因为Transformer要保持对每个输入token的分布式表征，否则模型就会记住这个[mask]是token ’hairy‘。至于单词带来的负面影响，因为一个单词被随机替换掉的概率只有15%*10% =1.5%，这个负面影响其实是可以忽略不计的。

另外文章指出每次只预测15%的单词，因此模型收敛的比较慢。

### 5.2 Next Sentence Prediction

Next Sentence Prediction(NSP)的任务是判断句子B是否是句子A的下文。如果是的话输出'IsNext',否则输出‘NotNext’。训练数据的生成方式是从平行语料中随机抽取的连续两句话，其中50%保留抽取的两句话，它们符合IsNext的关系，另外50%的第二句话是随机从语料中提取的，他们的关系是NotNext。这个关系保存在[CLS]符号中。


## 6. 微调
对NSP任务来说，其条件概率表示为$P=softmax(CW^T)$，其中，$C$是BERT输出中的[CLS]符号,$W$是可学习的权重矩阵。对于其它任务来说，我们也可以根据BERT的输出信息作出对应的预测。只需要在BERT的基础上再添加一个输出层便可以完成对特定任务的微调。

<img src="https://pic2.zhimg.com/80/v2-f576d9d19c9dcac1c6ee6ea28ea7a2d9_720w.jpg" width="500" />

微调的任务包括 \
（a）基于句子对的分类任务：

* MNLI：给定一个前提 (Premise) ，根据这个前提去推断假设 (Hypothesis) 与前提的关系。该任务的关系分为三种，蕴含关系 (Entailment)、矛盾关系 (Contradiction) 以及中立关系 (Neutral)。所以这个问题本质上是一个分类问题，我们需要做的是去发掘前提和假设这两个句子对之间的交互信息。
* QQP：基于Quora，判断 Quora 上的两个问题句是否表示的是一样的意思。
* QNLI：用于判断文本是否包含问题的答案，类似于我们做阅读理解定位问题所在的段落。
* STS-B：预测两个句子的相似性，包括5个级别。
* MRPC：也是判断两个句子是否是等价的。
* RTE：类似于MNLI，但是只是对蕴含关系的二分类判断，而且数据集更小。
* SWAG：从四个句子中选择为可能为前句下文的那个。


（b）基于单个句子的分类任务

* SST-2：电影评价的情感分析。
* CoLA：句子语义判断，是否是可接受的（Acceptable）。
对于GLUE数据集的分类任务（MNLI，QQP，QNLI，SST-B，MRPC，RTE，SST-2，CoLA），BERT的微调方法是根据[CLS]标志生成一组特征向量 $|C|$ ，并通过一层全连接进行微调。损失函数根据任务类型自行设计，例如多分类的softmax或者二分类的sigmoid。

SWAG的微调方法与GLUE数据集类似，只不过其输出是四个可能选项的softmax：

<img src="https://www.zhihu.com/equation?tex=+P_i+%3D+%5Cfrac%7Be%5E%7BV%5Ccdot+C_i%7D%7D%7B%5Csum_%7Bj%3D1%7D%5E4+e%5E%7BV%5Ccdot+C_i%7D%7D+%5Ctag1" width="500" />

（c）问答任务

SQuAD v1.1：给定一个句子（通常是一个问题）和一段描述文本，输出这个问题的答案，类似于做阅读理解的简答题。如图5.(c)表示的，SQuAD的输入是问题和描述文本的句子对。输出是特征向量，通过在描述文本上接一层激活函数为softmax的全连接来获得输出文本的条件概率，全连接的输出节点个数是语料中Token的个数。

<img src="https://www.zhihu.com/equation?tex=+P_i+%3D+%5Cfrac%7Be%5E%7BS%5Ccdot+T_i%7D%7D%7B%5Csum_%7Bj%7D+e%5E%7BS%5Ccdot+T_i%7D%7D+%5Ctag2" width="500" />

（d）命名实体识别

CoNLL-2003 NER：判断一个句子中的单词是不是Person，Organization，Location，Miscellaneous或者other（无命名实体）。微调CoNLL-2003 NER时将整个句子作为输入，在每个时间片输出一个概率，并通过softmax得到这个Token的实体类别。










